<a href="https://colab.research.google.com/github/shaikadish/imdbProject/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/GitHub/IMDB_project/imdbProject

/content/drive/MyDrive/GitHub/IMDB_project/imdbProject


In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
import selenium
import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd #Using panda to create our dataframe

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
# Making dictionary of links to movie pages


initial_link_dictionary={'movie_title':[],'movie_link':[],'year':[]}
# Feature Film/TV Movie, Released between 2010-01-01 and 2022-01-01, English, Including Adult Titles (Sorted by Year Ascending)
url = f"https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=2010-01-01,2022-01-01&languages=en&adult=include&sort=release_date,asc&start={0}&ref_=adv_nxt"
driver.get(url) 

for page_idx in range(0,1700):
  # Each loop collects the URL's for the movies on that page

  #Grab the block of each individual movie
  block = driver.find_elements_by_class_name('lister-item')
  #Set up for loop to run through all 50 movies on page
  for i in range(0,25):
      
      #Extracting title
      ftitle = block[i].find_element_by_class_name('lister-item-header').text
      #The extracted title has extra elements, so we will have to do some cleaning
      
      #Remove the order in front of the title
      forder = block[i].find_element_by_class_name('lister-item-index').text
      #Extract the year at the end
      fyear = block[i].find_element_by_class_name('lister-item-year').text
      #Drop the order and year and only keep the movie's name
      ftitle=ftitle.replace(" "+fyear,"")
      ftitle = ftitle.replace(forder+' ', '')
      #Then extract the link with cleaned title
      flink = block[i].find_element_by_link_text(ftitle).get_attribute('href')
  #Add item to the respective lists
      initial_link_dictionary['movie_title'].append(ftitle)
      initial_link_dictionary['year'].append(fyear[1:5])
      initial_link_dictionary['movie_link'].append(flink)
    
  # Go to next page
  load_more = driver.find_element_by_class_name('next-page')
  #click on that button
  load_more.click()
  
movie_url_df=pd.DataFrame(data=initial_link_dictionary)
movie_url_df.to_csv('movie_urls.csv')


In [ ]:
# get url for user review pages for each movie. Necessary because these links are not accessible from movie browsing page

#Set an empty list to store user review link
final_link_dictionary={'movie_title':[],'movie_link':[],'review_link':[],'year':[]}
for (url,movie_title,year) in zip(initial_link_dictionary['movie_link'],initial_link_dictionary['movie_title'],initial_link_dictionary['year']):
    try:
      #setup user agent for BS4, except some rare case, it would be the same for most browser 
      user_agent = {'User-Agent': 'Requests'}
      #Use request.get to load the whole page
      response = requests.get(url, headers = user_agent)
      #Parse the requests object to BS4 to transform it into html structure
      soup = BeautifulSoup(response.text, 'html.parser')
      #Find the link marked by the USER REVIEWS link text.
      review_link=url.split('/')
      review_link[-1]=soup.find('a', text = 'User reviews').get('href')
      review_link='/'.join(review_link) # This is a link to the page of all user reviews for a movie
      #Append the newly grabed link into its list
      #user_review_links.append(review_link)
      final_link_dictionary['movie_title'].append(movie_title)
      final_link_dictionary['movie_link'].append(url)
      final_link_dictionary['year'].append(year)
      final_link_dictionary['review_link'].append(review_link)
    except:
      continue # For movies with no written reviews

review_url_df=pd.DataFrame(data=final_link_dictionary)
review_url_df.to_csv('review_urls.csv')

In [ ]:
link_df=pd.review_url_df
link_df

,title,link,year,review_link
0,Tour de Force,https://www.imdb.com/title/tt1242787/?ref_=adv...,(2010),https://www.imdb.com/title/tt1242787/reviews/?...
1,Apocalypse Man,https://www.imdb.com/title/tt1663972/?ref_=adv...,(2010 TV Movie),https://www.imdb.com/title/tt1663972/reviews/?...
2,Welcome to the Rileys,https://www.imdb.com/title/tt1183923/?ref_=adv...,(2010),https://www.imdb.com/title/tt1183923/reviews/?...
3,Drones,https://www.imdb.com/title/tt1232775/?ref_=adv...,(2010),https://www.imdb.com/title/tt1232775/reviews/?...
4,Britain's Really Disgusting Food,https://www.imdb.com/title/tt1734143/?ref_=adv...,(2010 TV Movie),https://www.imdb.com/title/tt1734143/reviews/?...
...,...,...,...,...
409,Different Drum,https://www.imdb.com/title/tt3097128/?ref_=adv...,(2013),https://www.imdb.com/title/tt3097128/reviews/?...
410,Bigfoot Chronicles,https://www.imdb.com/title/tt3158728/?ref_=adv...,(2013),https://www.imdb.com/title/tt3158728/reviews/?...
411,Unlisted Owner,https://www.imdb.com/title/tt3522198/?ref_=adv...,(2013),https://www.imdb.com/title/tt3522198/reviews/?...
412,Not Another Sci-Fi Movie,https://www.imdb.com/title/tt4796190/?ref_=adv...,(2013),https://www.imdb.com/title/tt4796190/reviews/?...


In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
#Set list for each element:
title = []
content = []
rating = []
date = []
user_name = []
movie_title=[]
# Step 2, we will grab the data from each user review page
# Use Selenium to go to each user review page. each page is for a different film
for i in range(len(link_df['review_link'])):
  
  driver.get(link_df['review_link'][i]) # Go to user review page
  #driver.implicitly_wait(1) # tell the webdriver to wait for 1 seconds for the page to load to prevent blocked by anti spam software
  
  current_title=link_df['title'][i]

  # LOAD MORE REVIEWS FOR A GIVEN FILM. once loaded, can save reviews
  # Set up action to click on 'load more' button
  # note that each page on imdb has 25 reviews
  page = 1 #Set initial variable for while loop
  #We want at least 1000 review, so get 50 at a safe number
  while page<3:  
      try:
          #find the load more button on the webpage
          load_more = driver.find_element_by_id('load-more-trigger')
          #click on that button
          load_more.click()
          page+=1 #move on to next loadmore button
      except:
          #If couldnt find any button to click, stop
          break
  # After fully expand the page, we will grab data from whole website
  review = driver.find_elements_by_class_name('review-container')

  # save reviews for a given film
  for n in range(0,75):
      try:
          #Some reviewers only give review text or rating without the other, 
          #so we use try/except here to make sure each block of content must has all the element before append them to the list

          #Check if each review has all the elements
          ftitle = review[n].find_element_by_class_name('title').text
          #For the review content, some of them are hidden as spoiler, 
          #so we use the attribute 'textContent' here after extracting the 'content' tag
          fcontent = review[n].find_element_by_class_name('content').get_attribute("textContent").strip()
          frating = review[n].find_element_by_class_name('rating-other-user-rating').text
          fdate = review[n].find_element_by_class_name('review-date').text
          fname = review[n].find_element_by_class_name('display-name-link').text


          #Then add them to the respective list
          title.append(ftitle)
          content.append(fcontent)
          rating.append(frating.split('/')[0])
          date.append(fdate)
          user_name.append(fname)
          movie_title.append(current_title)
      except:
          continue

#Build data dictionary for dataframe
data = {'User_name': user_name, 
    'Review title': title, 
    'Review Rating': rating,
    'Review date' : date,
    'Review_body' : content,
    'Movie title': movie_title
    }
#Build dataframe for each movie to export
review = pd.DataFrame(data = data)
#movie = top50['Movie_name'][i] #grab the movie name from the top50 list    
#review['Movie_name'] = movie #create new column with the same movie name column    
#review.to_csv(f'data/{folder_name}/{i+1}.csv') #store them into individual file for each movies, so we can combine or check them later

In [ ]:
review.to_csv(f'review_data.csv')

In [ ]:
sum=0
for i in range(len(review['Review Rating'])):
  sum+=int(review['Review Rating'][i].split('/')[0])
average_score=sum/len(review['Review Rating'])
print(average_score)

6.246926497515041


In [ ]:
review['Review_body'][83]

"7 October 2016. Yes, this movie is predictable, the storyline nothing special that hasn't already been done before. Yet, the music is really sharp, crisp, and engrossingly fun and hip. The performances are subtly different and more natural and easy-going without overly drama. This Disney Version of the famous pop star accidentally meets unknown, plain girl is done in an appealing and straightforward, honest way that doesn't depend on anything more than a good music track, sung well, and performances and chemistry which are entertaining and smartly done. Other fun movies include I'll Be There (2003), A Royal Night Out (2015), Roman Holiday (1953), Music and Lyrics (2007), Pride and Prejudice (2005), The Artist (2011), The Devil Wears Prada (2006), Good Morning Call (2016),\n                \n                    1 out of 1 found this helpful.\n                        \n                            Was this review helpful?  Sign in to vote.\n                        \n                     